In [976]:
import numpy as np
import pandas as pd

from scipy.stats import wilcoxon

In [977]:
splits = {
    'dtc': pd.read_csv('splits_dtc.csv'),
    'dtr': pd.read_csv('splits_dtr.csv'),
    'rfc': pd.read_csv('splits_rfc.csv'),
    'rfr': pd.read_csv('splits_rfr.csv')
}
evaluations = {
    'dtc': pd.read_csv('evaluation_dtc.csv'),
    'dtr': pd.read_csv('evaluation_dtr.csv'),
    'rfc': pd.read_csv('evaluation_rfc.csv'),
    'rfr': pd.read_csv('evaluation_rfr.csv')
}

In [978]:
splits['dtc'] = splits['dtc'].rename(columns={'Unnamed: 0': 'name'})
splits['dtr'] = splits['dtr'].rename(columns={'Unnamed: 0': 'name'})
splits['rfc'] = splits['rfc'].rename(columns={'Unnamed: 0': 'name'})
splits['rfr'] = splits['rfr'].rename(columns={'Unnamed: 0': 'name'})

In [979]:
splits['dtc']['rate'] = splits['dtc']['n_lattice_splits'] / splits['dtc']['n_splits']
splits['dtr']['rate'] = splits['dtr']['n_lattice_splits'] / splits['dtr']['n_splits']
splits['rfc']['rate'] = splits['rfc']['n_lattice_splits'] / splits['rfc']['n_splits']
splits['rfr']['rate'] = splits['rfr']['n_lattice_splits'] / splits['rfr']['n_splits']

splits['dtc']['rate_kfold'] = splits['dtc']['n_lattice_splits_kfold'] / splits['dtc']['n_splits_kfold']
splits['dtr']['rate_kfold'] = splits['dtr']['n_lattice_splits_kfold'] / splits['dtr']['n_splits_kfold']
splits['rfc']['rate_kfold'] = splits['rfc']['n_lattice_splits_kfold'] / splits['rfc']['n_splits_kfold']
splits['rfr']['rate_kfold'] = splits['rfr']['n_lattice_splits_kfold'] / splits['rfr']['n_splits_kfold']

In [980]:
for scenario in ['dtc', 'dtr', 'rfc', 'rfr']:
    evaluations[scenario] = evaluations[scenario][evaluations[scenario]['mode'].isin(['<', '<='])]

In [981]:
for scenario in ['dtc', 'rfc']:
    evaluations[scenario] = evaluations[scenario]\
        .groupby(['name', 'mode'])\
        .apply(lambda pdf: pdf.sort_values('fold')['auc'].values.tolist())\
        .reset_index(drop=False)\
        .rename(columns={0: 'auc'})
for scenario in ['dtr', 'rfr']:
    evaluations[scenario] = evaluations[scenario]\
        .groupby(['name', 'mode'])\
        .apply(lambda pdf: pdf.sort_values('fold')['r2'].values.tolist())\
        .reset_index(drop=False)\
        .rename(columns={0: 'r2'})

In [982]:
def evaluate(pdf, score):
    leq = pdf[pdf['mode'] == '<='].iloc[0]
    l = pdf[pdf['mode'] == '<'].iloc[0]
    return pd.Series({
        'mean_leq': np.mean(leq[score]),
        'mean_l': np.mean(l[score]),
        'median_leq': np.median(leq[score]),
        'median_l': np.median(l[score]),
        'p': min(wilcoxon(leq[score], l[score], zero_method='zsplit', alternative='less').pvalue,
                 wilcoxon(leq[score], l[score], zero_method='zsplit', alternative='greater').pvalue)
    })

def evaluate2(pdf, score):
    leq = pdf[pdf['mode'] == '<='].iloc[0]
    l = pdf[pdf['mode'] == '<'].iloc[0]
    return pd.Series({
        'diff': np.mean(leq[score]) - np.mean(l[score]),
        #'mean_l': np.mean(l[score]),
        #'median_leq': np.median(leq[score]),
        #'median_l': np.median(l[score]),
        'p': min(wilcoxon(leq[score], l[score], zero_method='zsplit', alternative='less').pvalue,
                 wilcoxon(leq[score], l[score], zero_method='zsplit', alternative='greater').pvalue)
    })


In [983]:
dtc = evaluations['dtc'].groupby('name').apply(lambda x: evaluate2(x, 'auc'))
rfc = evaluations['rfc'].groupby('name').apply(lambda x: evaluate2(x, 'auc'))
dtr = evaluations['dtr'].groupby('name').apply(lambda x: evaluate2(x, 'r2'))
rfr = evaluations['rfr'].groupby('name').apply(lambda x: evaluate2(x, 'r2'))

In [984]:
dtc = pd.merge(dtc.reset_index(drop=False), splits['dtc'][['name', 'rate', 'rate_kfold']], on=['name']).set_index('name')
dtr = pd.merge(dtr.reset_index(drop=False), splits['dtr'][['name', 'rate', 'rate_kfold']], on=['name']).set_index('name')
rfc = pd.merge(rfc.reset_index(drop=False), splits['rfc'][['name', 'rate', 'rate_kfold']], on=['name']).set_index('name')
rfr = pd.merge(rfr.reset_index(drop=False), splits['rfr'][['name', 'rate', 'rate_kfold']], on=['name']).set_index('name')

In [985]:
dtc.columns = pd.MultiIndex.from_tuples(('Decision Tree', col) for col in dtc.columns)
rfc.columns = pd.MultiIndex.from_tuples(('Random Forest', col) for col in rfc.columns)

dtr.columns = pd.MultiIndex.from_tuples(('Decision Tree', col) for col in dtr.columns)
rfr.columns = pd.MultiIndex.from_tuples(('Random Forest', col) for col in rfr.columns)

In [986]:
binclas = pd.merge(dtc, rfc, left_index=True, right_index=True).reset_index(drop=False)
regr = pd.merge(dtr, rfr, left_index=True, right_index=True).reset_index(drop=False)

In [987]:
binclas.columns = pd.MultiIndex.from_tuples(('Classification', *col) for col in binclas.columns)
regr.columns = pd.MultiIndex.from_tuples(('Regression', *col) for col in regr.columns)

In [988]:
binclas

Classification                                                   \
                      name Decision Tree                                      
                                    diff             p      rate rate_kfold   
0              abalone9_18      0.000000  5.000000e-01  0.000000   0.000000   
1             appendicitis      0.000000  5.000000e-01  0.000000   0.003534   
2                     bupa      0.000105  7.051001e-03  0.117647   0.095642   
3         cleveland-0_vs_4     -0.003139  7.374231e-02  0.000000   0.014337   
4                   ecoli1      0.000000  5.000000e-01  0.000000   0.056667   
5                   glass0      0.000000  5.000000e-01  0.000000   0.032316   
6                 haberman     -0.000335  4.971797e-04  0.000000   0.020378   
7                hepatitis     -0.000244  4.734861e-01  0.000000   0.044077   
8             mammographic      0.000026  3.769766e-03  0.000000   0.001086   
9                   monk-2      0.000000  5.000000e-01  0.000000   0.000000   
10            new_thyroid1      0.000004  4.119086e-01  0.000000   0.014851   
11    page-blocks-1-3_vs_4      0.000149  1.203289e-10  0.250000   0.116505   
12                    pima     -0.000307  8.445611e-05  0.000000   0.049003   
13            poker-9_vs_7      0.000000  5.000000e-01  0.000000   0.000000   
14                 saheart     -0.000008  4.050898e-02  0.000000   0.026832   
15        shuttle-6_vs_2-3      0.000000  5.000000e-01  0.000000   0.000000   
16  winequality-red-3_vs_5      0.000000  5.000000e-01  0.000000   0.000000   
17               wisconsin     -0.000099  1.070315e-13  0.000000   0.027778   
18    yeast-0-3-5-9_vs_7-8     -0.000044  4.123225e-01  0.000000   0.041905   

                                                     
   Random Forest                                     
            diff             p      rate rate_kfold  
0  -1.271275e-06  4.486313e-01  0.082214   0.076214  
1  -4.411765e-06  3.965131e-01  0.022556   0.031279  
2   2.388147e-03  6.398291e-56  0.249033   0.243139  
3  -7.637311e-04  3.434543e-06  0.094907   0.106972  
4  -6.760699e-05  1.832211e-04  0.117832   0.122796  
5  -1.999032e-05  2.266212e-01  0.049146   0.045386  
6   5.616830e-06  4.816973e-01  0.039381   0.052000  
7  -2.828571e-04  2.330632e-03  0.034364   0.042390  
8   5.507317e-07  3.510752e-03  0.018786   0.036491  
9   0.000000e+00  5.000000e-01  0.000551   0.002608  
10  7.539683e-05  3.462011e-02  0.071930   0.090939  
11 -4.141130e-06  4.121740e-01  0.060357   0.061899  
12  3.976240e-06  3.692187e-01  0.162217   0.188265  
13 -1.736259e-03  2.718275e-11  0.142454   0.163727  
14  5.643784e-05  2.652175e-02  0.073353   0.078482  
15  0.000000e+00  4.295878e-01  0.135693   0.131752  
16 -4.330856e-04  7.329705e-09  0.068783   0.063618  
17  3.931485e-05  1.111545e-01  0.091975   0.116448  
18 -2.248626e-04  6.460400e-08  0.091002   0.089733

In [989]:
result = pd.concat([binclas, regr], axis=1)

In [990]:
result

Classification                                                   \
                      name Decision Tree                                      
                                    diff             p      rate rate_kfold   
0              abalone9_18      0.000000  5.000000e-01  0.000000   0.000000   
1             appendicitis      0.000000  5.000000e-01  0.000000   0.003534   
2                     bupa      0.000105  7.051001e-03  0.117647   0.095642   
3         cleveland-0_vs_4     -0.003139  7.374231e-02  0.000000   0.014337   
4                   ecoli1      0.000000  5.000000e-01  0.000000   0.056667   
5                   glass0      0.000000  5.000000e-01  0.000000   0.032316   
6                 haberman     -0.000335  4.971797e-04  0.000000   0.020378   
7                hepatitis     -0.000244  4.734861e-01  0.000000   0.044077   
8             mammographic      0.000026  3.769766e-03  0.000000   0.001086   
9                   monk-2      0.000000  5.000000e-01  0.000000   0.000000   
10            new_thyroid1      0.000004  4.119086e-01  0.000000   0.014851   
11    page-blocks-1-3_vs_4      0.000149  1.203289e-10  0.250000   0.116505   
12                    pima     -0.000307  8.445611e-05  0.000000   0.049003   
13            poker-9_vs_7      0.000000  5.000000e-01  0.000000   0.000000   
14                 saheart     -0.000008  4.050898e-02  0.000000   0.026832   
15        shuttle-6_vs_2-3      0.000000  5.000000e-01  0.000000   0.000000   
16  winequality-red-3_vs_5      0.000000  5.000000e-01  0.000000   0.000000   
17               wisconsin     -0.000099  1.070315e-13  0.000000   0.027778   
18    yeast-0-3-5-9_vs_7-8     -0.000044  4.123225e-01  0.000000   0.041905   

                                                                Regression  \
   Random Forest                                                      name   
            diff             p      rate rate_kfold                          
0  -1.271275e-06  4.486313e-01  0.082214   0.076214                airfoil   
1  -4.411765e-06  3.965131e-01  0.022556   0.031279               autoMPG6   
2   2.388147e-03  6.398291e-56  0.249033   0.243139               baseball   
3  -7.637311e-04  3.434543e-06  0.094907   0.106972        cpu_performance   
4  -6.760699e-05  1.832211e-04  0.117832   0.122796           daily-demand   
5  -1.999032e-05  2.266212e-01  0.049146   0.045386               diabetes   
6   5.616830e-06  4.816973e-01  0.039381   0.052000     excitation_current   
7  -2.828571e-04  2.330632e-03  0.034364   0.042390                  laser   
8   5.507317e-07  3.510752e-03  0.018786   0.036491   maternal_health_risk   
9   0.000000e+00  5.000000e-01  0.000551   0.002608           medical_cost   
10  7.539683e-05  3.462011e-02  0.071930   0.090939                 o-ring   
11 -4.141130e-06  4.121740e-01  0.060357   0.061899                plastic   
12  3.976240e-06  3.692187e-01  0.162217   0.188265  qsar-aquatic-toxicity   
13 -1.736259e-03  2.718275e-11  0.142454   0.163727  real_estate_valuation   
14  5.643784e-05  2.652175e-02  0.073353   0.078482                  servo   
15  0.000000e+00  4.295878e-01  0.135693   0.131752             slump_test   
16 -4.330856e-04  7.329705e-09  0.068783   0.063618                wankara   
17  3.931485e-05  1.111545e-01  0.091975   0.116448                wsn-ale   
18 -2.248626e-04  6.460400e-08  0.091002   0.089733    yacht_hydrodynamics   

                                                                    \
   Decision Tree                                     Random Forest   
            diff              p      rate rate_kfold          diff   
0   8.851710e-06   8.171774e-18  0.004765   0.007478 -9.730602e-06   
1  -3.084192e-04   8.534206e-04  0.086957   0.049645  3.079508e-05   
2  -1.287322e-03   3.215768e-01  0.086957   0.141608  6.889246e-04   
3   2.135930e-03   1.774716e-21  0.233333   0.206069  1.852589e-03   
4  -3.387756e-04   1.569446e-02  0.000000   0.008483 -7.188248e-04

In [991]:
def boldface(row):
    res = pd.Series()
    if row['p'] < -1:
        for label, _ in row.items():
            if label != 'p':
                value = row[label]
                value = f'{value:.3f}'
                if value[0] == '0':
                    value = value[1:]
                else:
                    value = '1'
                res[label] = f'{{ \\bfseries {value}}}'
    else:
        for label, _ in row.items():
            if label != 'p' and not label.startswith('rate'):
                value = row[label]
                if abs(value) > 1e-9:
                    value = f'{value:.0e}'
                else:
                    value = '0'
                #if value[0] == '0' or value[0] == '-':
                #    value = value[1:]
                #else:
                #    value = '1'
                if value[0] != '-':
                    value = ' ' + value
                res[label] = f'{value}'
            elif label.startswith('rate'):
                value = row[label]
                res[label] = f'{value:.2f}'
                res[label] = res[label][1:]
    res['p'] = row['p']
    return res

In [992]:
result[('Classification', 'Decision Tree')] = result[('Classification', 'Decision Tree')].apply(boldface, axis=1)
result[('Classification', 'Random Forest')] = result[('Classification', 'Random Forest')].apply(boldface, axis=1)
result[('Regression', 'Decision Tree')] = result[('Regression', 'Decision Tree')].apply(boldface, axis=1)
result[('Regression', 'Random Forest')] = result[('Regression', 'Random Forest')].apply(boldface, axis=1)

/tmp/ipykernel_1590836/3492045352.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Decision Tree')] = result[('Classification', 'Decision Tree')].apply(boldface, axis=1)
/tmp/ipykernel_1590836/3492045352.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Decision Tree')] = result[('Classification', 'Decision Tree')].apply(boldface, axis=1)
/tmp/ipykernel_1590836/3492045352.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Random Forest')] = result[('Classification', 'Random Forest')].apply(boldface, axis=1)
/tmp/ipykernel_1590836/3492045352.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Random Forest')] = result[('Classification', 'Random Forest')].apply(boldface, axis=1)
/tmp/ipykernel_1590836/3492045352.py:3: PerformanceWarning: indexing past lexsort depth 

In [993]:
result[('Classification', 'Decision Tree', 'p')] = result[('Classification', 'Decision Tree', 'p')].apply(lambda x: '$\\neq$' if x < 0.05 else '')
result[('Classification', 'Random Forest', 'p')] = result[('Classification', 'Random Forest', 'p')].apply(lambda x: '$\\neq$' if x < 0.05 else '')
result[('Regression', 'Decision Tree', 'p')] = result[('Regression', 'Decision Tree', 'p')].apply(lambda x: '$\\neq$' if x < 0.05 else '')
result[('Regression', 'Random Forest', 'p')] = result[('Regression', 'Random Forest', 'p')].apply(lambda x: '$\\neq$' if x < 0.05 else '')

In [994]:
result

Classification                                        \
                      name Decision Tree                           
                                    diff       p rate rate_kfold   
0              abalone9_18             0          .00        .00   
1             appendicitis             0          .00        .00   
2                     bupa         1e-04  $\neq$  .12        .10   
3         cleveland-0_vs_4        -3e-03          .00        .01   
4                   ecoli1             0          .00        .06   
5                   glass0             0          .00        .03   
6                 haberman        -3e-04  $\neq$  .00        .02   
7                hepatitis        -2e-04          .00        .04   
8             mammographic         3e-05  $\neq$  .00        .00   
9                   monk-2             0          .00        .00   
10            new_thyroid1         4e-06          .00        .01   
11    page-blocks-1-3_vs_4         1e-04  $\neq$  .25        .12   
12                    pima        -3e-04  $\neq$  .00        .05   
13            poker-9_vs_7             0          .00        .00   
14                 saheart        -8e-06  $\neq$  .00        .03   
15        shuttle-6_vs_2-3             0          .00        .00   
16  winequality-red-3_vs_5             0          .00        .00   
17               wisconsin        -1e-04  $\neq$  .00        .03   
18    yeast-0-3-5-9_vs_7-8        -4e-05          .00        .04   

                                                     Regression                \
   Random Forest                                           name Decision Tree   
            diff       p rate rate_kfold                                 diff   
0         -1e-06          .08        .08                airfoil         9e-06   
1         -4e-06          .02        .03               autoMPG6        -3e-04   
2          2e-03  $\neq$  .25        .24               baseball        -1e-03   
3         -8e-04  $\neq$  .09        .11        cpu_performance         2e-03   
4         -7e-05  $\neq$  .12        .12           daily-demand        -3e-04   
5         -2e-05          .05        .05               diabetes        -7e-05   
6          6e-06          .04        .05     excitation_current        -4e-08   
7         -3e-04  $\neq$  .03        .04                  laser        -9e-04   
8          6e-07  $\neq$  .02        .04   maternal_health_risk         1e-03   
9              0          .00        .00           medical_cost             0   
10         8e-05  $\neq$  .07        .09                 o-ring        -4e-03   
11        -4e-06          .06        .06                plastic         1e-03   
12         4e-06          .16        .19  qsar-aquatic-toxicity             0   
13        -2e-03  $\neq$  .14        .16  real_estate_valuation        -4e-04   
14         6e-05  $\neq$  .07        .08                  servo        -7e-06   
15             0          .14        .13             slump_test         5e-04   
16        -4e-04  $\neq$  .07        .06                wankara        -5e-05   
17         4e-05          .09        .12                wsn-ale        -1e-03   
18        -2e-04  $\neq$  .09        .09    yacht_hydrodynamics             0   

                                                                  
                           Random Forest                          
         p rate rate_kfold          diff       p rate rate_kfold  
0   $\neq$  .00        .01        -1e-05  $\neq$  .01        .01  
1   $\neq$  .09        .05         3e-05          .18        .17  
2           .09        .14         7e-04  $\neq$  .23        .24  
3   $\neq$  .23        .21         2e-03  $\neq$  .24        .22  
4   $\neq$  .00        .01        -7e-04  $\neq$  .03        .03  
5           .00        .02         7e-04  $\neq$  .00        .00  
6   $\neq$  .10        .10         2e-07  $\neq$  .13        .12  
7   $\neq$  .25        .28        -2e-04  $\neq$  .38        .

In [995]:
#final = result.drop(columns=[('Classification', 'Decision Tree', 'p'), ('Classification', 'Random Forest', 'p'),
#                                ('Regression', 'Decision Tree', 'p'), ('Regression', 'Random Forest', 'p')])
final = result

In [996]:
final

Classification                                        \
                      name Decision Tree                           
                                    diff       p rate rate_kfold   
0              abalone9_18             0          .00        .00   
1             appendicitis             0          .00        .00   
2                     bupa         1e-04  $\neq$  .12        .10   
3         cleveland-0_vs_4        -3e-03          .00        .01   
4                   ecoli1             0          .00        .06   
5                   glass0             0          .00        .03   
6                 haberman        -3e-04  $\neq$  .00        .02   
7                hepatitis        -2e-04          .00        .04   
8             mammographic         3e-05  $\neq$  .00        .00   
9                   monk-2             0          .00        .00   
10            new_thyroid1         4e-06          .00        .01   
11    page-blocks-1-3_vs_4         1e-04  $\neq$  .25        .12   
12                    pima        -3e-04  $\neq$  .00        .05   
13            poker-9_vs_7             0          .00        .00   
14                 saheart        -8e-06  $\neq$  .00        .03   
15        shuttle-6_vs_2-3             0          .00        .00   
16  winequality-red-3_vs_5             0          .00        .00   
17               wisconsin        -1e-04  $\neq$  .00        .03   
18    yeast-0-3-5-9_vs_7-8        -4e-05          .00        .04   

                                                     Regression                \
   Random Forest                                           name Decision Tree   
            diff       p rate rate_kfold                                 diff   
0         -1e-06          .08        .08                airfoil         9e-06   
1         -4e-06          .02        .03               autoMPG6        -3e-04   
2          2e-03  $\neq$  .25        .24               baseball        -1e-03   
3         -8e-04  $\neq$  .09        .11        cpu_performance         2e-03   
4         -7e-05  $\neq$  .12        .12           daily-demand        -3e-04   
5         -2e-05          .05        .05               diabetes        -7e-05   
6          6e-06          .04        .05     excitation_current        -4e-08   
7         -3e-04  $\neq$  .03        .04                  laser        -9e-04   
8          6e-07  $\neq$  .02        .04   maternal_health_risk         1e-03   
9              0          .00        .00           medical_cost             0   
10         8e-05  $\neq$  .07        .09                 o-ring        -4e-03   
11        -4e-06          .06        .06                plastic         1e-03   
12         4e-06          .16        .19  qsar-aquatic-toxicity             0   
13        -2e-03  $\neq$  .14        .16  real_estate_valuation        -4e-04   
14         6e-05  $\neq$  .07        .08                  servo        -7e-06   
15             0          .14        .13             slump_test         5e-04   
16        -4e-04  $\neq$  .07        .06                wankara        -5e-05   
17         4e-05          .09        .12                wsn-ale        -1e-03   
18        -2e-04  $\neq$  .09        .09    yacht_hydrodynamics             0   

                                                                  
                           Random Forest                          
         p rate rate_kfold          diff       p rate rate_kfold  
0   $\neq$  .00        .01        -1e-05  $\neq$  .01        .01  
1   $\neq$  .09        .05         3e-05          .18        .17  
2           .09        .14         7e-04  $\neq$  .23        .24  
3   $\neq$  .23        .21         2e-03  $\neq$  .24        .22  
4   $\neq$  .00        .01        -7e-04  $\neq$  .03        .03  
5           .00        .02         7e-04  $\neq$  .00        .00  
6   $\neq$  .10        .10         2e-07  $\neq$  .13        .12  
7   $\neq$  .25        .28        -2e-04  $\neq$  .38        .

In [997]:
final.columns[0]

('Classification', 'name', '')

In [998]:
final[('Classification', 'name', '')] = final[('Classification', 'name', '')].apply(lambda x: x.replace('_', '-'))
final[('Regression', 'name', '')] = final[('Regression', 'name', '')].apply(lambda x: x.replace('_', '-'))

In [999]:
def mode(string):
    if not string.startswith('rate'):
        first, second = string.split('_')
        first = 'a' if first == 'mean' else 'm'
        second = '<' if second == 'l' else '\leq'
        return first + '$_{' + second + '}$'
    else:
        if string == 'rate':
            return '$\\rho$'
        else:
            return '$\\rho_{k}$'

In [1000]:
final.columns = pd.MultiIndex.from_tuples([col if col[-1] in ['', 'p', 'diff'] else (col[0], col[1], mode(col[2])) for col in final.columns])

In [1001]:
final

Classification                                          \
                      name Decision Tree                             
                                    diff       p $\rho$ $\rho_{k}$   
0              abalone9-18             0            .00        .00   
1             appendicitis             0            .00        .00   
2                     bupa         1e-04  $\neq$    .12        .10   
3         cleveland-0-vs-4        -3e-03            .00        .01   
4                   ecoli1             0            .00        .06   
5                   glass0             0            .00        .03   
6                 haberman        -3e-04  $\neq$    .00        .02   
7                hepatitis        -2e-04            .00        .04   
8             mammographic         3e-05  $\neq$    .00        .00   
9                   monk-2             0            .00        .00   
10            new-thyroid1         4e-06            .00        .01   
11    page-blocks-1-3-vs-4         1e-04  $\neq$    .25        .12   
12                    pima        -3e-04  $\neq$    .00        .05   
13            poker-9-vs-7             0            .00        .00   
14                 saheart        -8e-06  $\neq$    .00        .03   
15        shuttle-6-vs-2-3             0            .00        .00   
16  winequality-red-3-vs-5             0            .00        .00   
17               wisconsin        -1e-04  $\neq$    .00        .03   
18    yeast-0-3-5-9-vs-7-8        -4e-05            .00        .04   

                                                       Regression  \
   Random Forest                                             name   
            diff       p $\rho$ $\rho_{k}$                          
0         -1e-06            .08        .08                airfoil   
1         -4e-06            .02        .03               autoMPG6   
2          2e-03  $\neq$    .25        .24               baseball   
3         -8e-04  $\neq$    .09        .11        cpu-performance   
4         -7e-05  $\neq$    .12        .12           daily-demand   
5         -2e-05            .05        .05               diabetes   
6          6e-06            .04        .05     excitation-current   
7         -3e-04  $\neq$    .03        .04                  laser   
8          6e-07  $\neq$    .02        .04   maternal-health-risk   
9              0            .00        .00           medical-cost   
10         8e-05  $\neq$    .07        .09                 o-ring   
11        -4e-06            .06        .06                plastic   
12         4e-06            .16        .19  qsar-aquatic-toxicity   
13        -2e-03  $\neq$    .14        .16  real-estate-valuation   
14         6e-05  $\neq$    .07        .08                  servo   
15             0            .14        .13             slump-test   
16        -4e-04  $\neq$    .07        .06                wankara   
17         4e-05            .09        .12                wsn-ale   
18        -2e-04  $\neq$    .09        .09    yacht-hydrodynamics   

                                                                         \
   Decision Tree                           Random Forest                  
            diff       p $\rho$ $\rho_{k}$          diff       p $\rho$   
0          9e-06  $\neq$    .00        .01        -1e-05  $\neq$    .01   
1         -3e-04  $\neq$    .09        .05         3e-05            .18   
2         -1e-03            .09        .14         7e-04  $\neq$    .23   
3          2e-03  $\neq$    .23        .21         2e-03  $\neq$    .24   
4         -3e-04  $\neq$    .00        .01        -7e-04  $\neq$    .03   
5         -7e-05            .00        .02         7e-04  $\neq$    .00   
6         -4e-08  $\neq$    .10        .10         2e-07  $\neq$    .13   
7         -9e-04  $\neq$    .25        .28        -2e-04  $\neq$    .38   
8          1e-03  $\neq$    .22        .23        -7e-04  $\neq$    .24   
9              0            .00        .00         2

In [1002]:
latex = final.to_latex(
    index=False,
    multicolumn_format='c'
)

In [1003]:
tabular_string = latex[len('\\begin{tabular}{'): len('\\begin{tabular}{') + len(final.columns)]
print(tabular_string)
tabular_string_new = 'lrlrrrlrrlrrrlrrrl'
updated = '@{\hspace{4pt}}'.join(tabular_string_new)
latex = latex.replace(tabular_string, updated)

llllllllllllllllll


In [1004]:
print(latex)

\begin{tabular}{l@{\hspace{4pt}}r@{\hspace{4pt}}l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}l}
\toprule
\multicolumn{9}{c}{Classification} & \multicolumn{9}{c}{Regression} \\
name & \multicolumn{4}{c}{Decision Tree} & \multicolumn{4}{c}{Random Forest} & name & \multicolumn{4}{c}{Decision Tree} & \multicolumn{4}{c}{Random Forest} \\
 & diff & p & $\rho$ & $\rho_{k}$ & diff & p & $\rho$ & $\rho_{k}$ &  & diff & p & $\rho$ & $\rho_{k}$ & diff & p & $\rho$ & $\rho_{k}$ \\
\midrule
abalone9-18 &  0 &  & .00 & .00 & -1e-06 &  & .08 & .08 & airfoil &  9e-06 & $\neq$ & .00 & .01 & -1e-05 & $\neq$ & .01 & .01 \\
appendicitis &  0 &  & .00 & .00 & -4e-06 &  & .02 & .03 & autoMPG6 & -3e-04 & $\neq$ & .09 & .05 &  3e-05 &  & .18 & .17 \\
bupa &  1e-04 & $\neq$ & .12 & .10 &  2e-03 & $\neq$ & .25 & .24 & b

In [1005]:
with open('tab_presence.tex', 'wt') as file:
    file.write(latex)

In [1006]:
binclas['Decision Tree']

KeyError: 'Decision Tree'

In [ ]:
scenario = 'rfc'

In [ ]:
data = pd.read_csv(f'evaluation_{scenario}.csv')

In [ ]:
score = 'auc' if 'auc' in data.columns else 'r2'

In [ ]:
def do_testing(pdf, metric):
    params = pdf['mode'].drop_duplicates().values
    pdf0 = pdf[pdf['mode'] == params[0]].sort_values('fold')
    pdf1 = pdf[pdf['mode'] == params[1]].sort_values('fold')

    return pd.Series({
        'wilcoxon': wilcoxon(pdf0[metric], pdf1[metric], zero_method='zsplit').pvalue
    })

In [ ]:
grouped = data\
    .groupby(['name', 'mode'])\
    .apply(lambda pdf: pdf.sort_values('fold')[score].values.tolist())\
    .reset_index(drop=False)\
    .rename(columns={0: score})

In [ ]:
def evaluate_one(grouped, pivot_postfix='<='):
    pivot_row = grouped[grouped['mode'] == f"{pivot_postfix}"].iloc[0]
    pivot_label = pivot_row['mode']

    other_rows = grouped[grouped['mode'] != pivot_label]

    result = {f'{score}_{pivot_label}': np.mean(pivot_row[score])}

    for idx, row in other_rows.iterrows():
        result[f'{score}_{row["mode"]}'] = np.mean(row[score])
        if row["mode"] in ['<', '<=']:
            result[f'p_{row["mode"]}'] = wilcoxon(pivot_row[score], row[score], zero_method="zsplit").pvalue
        else:
            result[f'p_{row["mode"]}_l'] = wilcoxon(pivot_row[score], row[score], zero_method="zsplit", alternative="less").pvalue
            result[f'p_{row["mode"]}_g'] = wilcoxon(pivot_row[score], row[score], zero_method="zsplit", alternative="greater").pvalue

    return pd.Series(result)

def evaluate_min_max(grouped):
    pivot_leq = grouped[grouped['mode'] == "<="].iloc[0]
    pivot_l = grouped[grouped['mode'] == "<"].iloc[0]

    score_leq = np.mean(pivot_leq[score])
    score_l = np.mean(pivot_l[score])

    if score_leq < score_l:
        min_row = pivot_leq
        max_row = pivot_l
    else:
        min_row = pivot_l
        max_row = pivot_leq

    other_rows = grouped[~grouped['mode'].isin(['<=', '<'])]

    result = {f'{score}_min': np.mean(min_row[score]),
                f'{score}_max': np.mean(max_row[score])}

    for idx, row in other_rows.iterrows():
        result[f'{score}_{row["mode"]}'] = np.mean(row[score])

        result[f'p_{row["mode"]}_min'] = wilcoxon(row[score], min_row[score], zero_method="zsplit", alternative="greater").pvalue
        result[f'p_{row["mode"]}_max'] = wilcoxon(row[score], max_row[score], zero_method="zsplit", alternative="less").pvalue

    return pd.Series(result)

In [ ]:
def evaluate_all(data, pivot_postfix='<='):
    return data.groupby("name").apply(lambda x: evaluate_one(x, pivot_postfix))


In [ ]:
grouped.groupby("name").apply(evaluate_min_max)

          name      mode                                                auc
0  abalone9_18         <  [0.8679549114331723, 0.7581521739130435, 0.667...
1  abalone9_18        <=  [0.8679549114331723, 0.7581521739130435, 0.667...
2  abalone9_18   avg_all  [0.8679549114331723, 0.7581521739130435, 0.667...
3  abalone9_18  avg_half  [0.8679549114331723, 0.7581521739130435, 0.667...
4  abalone9_18  specific  [0.8679549114331723, 0.7581521739130435, 0.667...
           name      mode                                                auc
5  appendicitis         <  [0.8588235294117648, 0.7647058823529411, 0.911...
6  appendicitis        <=  [0.8588235294117648, 0.7647058823529411, 0.897...
7  appendicitis   avg_all  [0.8588235294117648, 0.7647058823529411, 0.897...
8  appendicitis  avg_half  [0.8588235294117648, 0.7647058823529411, 0.897...
9  appendicitis  specific  [0.8588235294117648, 0.7647058823529411, 0.897...
    name      mode                                                auc
10  bupa    

,auc_min,auc_max,auc_avg_all,p_avg_all_min,p_avg_all_max,auc_avg_half,p_avg_half_min,p_avg_half_max,auc_specific,p_specific_min,p_specific_max
name,,,,,,,,,,,
abalone9_18,0.850400,0.850401,0.850404,4.298141e-01,5.181608e-01,0.850399,1.061404e-01,8.676687e-01,0.850401,4.486313e-01,5.000000e-01
appendicitis,0.854646,0.854651,0.854668,5.348933e-01,5.705234e-01,0.854638,5.531214e-01,5.522187e-01,0.854646,5.000000e-01,6.034869e-01
bupa,0.768082,0.770470,0.769375,1.325996e-46,1.526726e-35,0.769292,1.486021e-33,1.121001e-30,0.769884,8.603254e-37,5.725887e-09
cleveland-0_vs_4,0.975759,0.976522,0.976118,5.246090e-03,4.020133e-03,0.976105,1.072252e-02,5.042984e-03,0.976522,3.434543e-06,5.000000e-01
ecoli1,0.958258,0.958325,0.958301,6.066381e-03,4.039756e-02,0.958288,4.709558e-02,5.118013e-03,0.958325,1.832211e-04,5.000000e-01
glass0,0.930402,0.930422,0.930414,3.445875e-01,3.779421e-01,0.930410,4.050638e-01,3.487645e-01,0.930410,3.776664e-01,3.288417e-01
haberman,0.720607,0.720613,0.720585,7.653497e-01,2.682971e-01,0.720602,4.698231e-01,3.223207e-01,0.720607,5.000000e-01,5.183027e-01
hepatitis,0.871735,0.872018,0.871873,6.593984e-02,2.887862e-02,0.871888,3.527014e-02,6.633195e-02,0.871780,2.706473e-01,9.879414e-03
lymphography,0.994233,0.995412,0.995306,1.308777e-03,4.129667e-01,0.995306,3.220631e-03,3.495543e-01,0.995133,2.424079e-02,2.973790e-01


In [ ]:
final = evaluate_all(grouped, pivot_postfix='<=')
final

,auc_<=,auc_<,p_<,auc_avg_all,p_avg_all_l,p_avg_all_g,auc_avg_half,p_avg_half_l,p_avg_half_g,auc_specific,p_specific_l,p_specific_g
name,,,,,,,,,,,,
abalone9_18,0.850400,0.850401,8.972626e-01,0.850404,4.298141e-01,5.701859e-01,0.850399,1.061404e-01,8.938596e-01,0.850401,4.486313e-01,5.513687e-01
appendicitis,0.854646,0.854651,7.930262e-01,0.854668,5.348933e-01,4.651067e-01,0.854638,5.531214e-01,4.468786e-01,0.854646,5.000000e-01,5.000000e-01
bupa,0.770470,0.768082,1.279658e-55,0.769375,1.000000e+00,1.526726e-35,0.769292,1.000000e+00,1.121001e-30,0.769884,1.000000e+00,5.725887e-09
cleveland-0_vs_4,0.975759,0.976522,6.869086e-06,0.976118,5.246090e-03,9.947539e-01,0.976105,1.072252e-02,9.892775e-01,0.976522,3.434543e-06,9.999966e-01
ecoli1,0.958258,0.958325,3.664422e-04,0.958301,6.066381e-03,9.939336e-01,0.958288,4.709558e-02,9.529044e-01,0.958325,1.832211e-04,9.998168e-01
glass0,0.930402,0.930422,4.532423e-01,0.930414,3.445875e-01,6.554125e-01,0.930410,4.050638e-01,5.949362e-01,0.930410,3.776664e-01,6.223336e-01
haberman,0.720613,0.720607,9.633946e-01,0.720585,7.317029e-01,2.682971e-01,0.720602,6.776793e-01,3.223207e-01,0.720607,4.816973e-01,5.183027e-01
hepatitis,0.871735,0.872018,4.661264e-03,0.871873,6.593984e-02,9.340602e-01,0.871888,3.527014e-02,9.647299e-01,0.871780,2.706473e-01,7.293527e-01
lymphography,0.994233,0.995412,1.399264e-02,0.995306,1.308777e-03,9.986912e-01,0.995306,3.220631e-03,9.967794e-01,0.995133,2.424079e-02,9.757592e-01


In [ ]:
(final <= 0.05).sum()

auc_<=          0
auc_<           0
p_<             9
auc_avg_all     0
p_avg_all_l     6
p_avg_all_g     1
auc_avg_half    0
p_avg_half_l    8
p_avg_half_g    2
auc_specific    0
p_specific_l    6
p_specific_g    3
dtype: int64

In [ ]:
final = evaluate_all(grouped, pivot_postfix='<')
final

,auc_<,auc_<=,p_<=,auc_avg_all,p_avg_all_l,p_avg_all_g,auc_avg_half,p_avg_half_l,p_avg_half_g,auc_specific,p_specific_l,p_specific_g
name,,,,,,,,,,,,
abalone9_18,0.850401,0.850400,8.972626e-01,0.850404,4.818392e-01,0.518161,0.850399,1.323313e-01,0.867669,0.850401,5.000000e-01,5.000000e-01
appendicitis,0.854651,0.854646,7.930262e-01,0.854668,4.294766e-01,0.570523,0.854638,4.477813e-01,0.552219,0.854646,3.965131e-01,6.034869e-01
bupa,0.768082,0.770470,1.279658e-55,0.769375,1.325996e-46,1.000000,0.769292,1.486021e-33,1.000000,0.769884,8.603254e-37,1.000000e+00
cleveland-0_vs_4,0.976522,0.975759,6.869086e-06,0.976118,9.959799e-01,0.004020,0.976105,9.949570e-01,0.005043,0.976522,5.000000e-01,5.000000e-01
ecoli1,0.958325,0.958258,3.664422e-04,0.958301,9.596024e-01,0.040398,0.958288,9.948820e-01,0.005118,0.958325,5.000000e-01,5.000000e-01
glass0,0.930422,0.930402,4.532423e-01,0.930414,6.220579e-01,0.377942,0.930410,6.512355e-01,0.348765,0.930410,6.711583e-01,3.288417e-01
haberman,0.720607,0.720613,9.633946e-01,0.720585,7.653497e-01,0.234650,0.720602,4.698231e-01,0.530177,0.720607,5.000000e-01,5.000000e-01
hepatitis,0.872018,0.871735,4.661264e-03,0.871873,9.711214e-01,0.028879,0.871888,9.336680e-01,0.066332,0.871780,9.901206e-01,9.879414e-03
lymphography,0.995412,0.994233,1.399264e-02,0.995306,5.870333e-01,0.412967,0.995306,6.504457e-01,0.349554,0.995133,7.026210e-01,2.973790e-01


In [ ]:
(final <= 0.05).sum()

auc_<           0
auc_<=          0
p_<=            9
auc_avg_all     0
p_avg_all_l     2
p_avg_all_g     7
auc_avg_half    0
p_avg_half_l    1
p_avg_half_g    5
auc_specific    0
p_specific_l    1
p_specific_g    2
dtype: int64